## Fake News Classifier Using Bidirectional LSTM

In [91]:
import pandas as pd

In [92]:
df = pd.read_csv('train.csv', engine='python', error_bad_lines = False)

C:\Users\USER\AppData\Local\Temp\ipykernel_22372\1413091467.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('train.csv', engine='python', error_bad_lines = False)
Skipping line 13535: field larger than field limit (131072)
Skipping line 19783: field larger than field limit (131072)
Skipping line 13541: Expected 5 fields in line 13541, saw 6
Skipping line 19782: Expected 5 fields in line 19782, saw 7
Skipping line 19783: Expected 5 fields in line 19783, saw 10
Skipping line 19785: Expected 5 fields in line 19785, saw 14
Skipping line 19787: Expected 5 fields in line 19787, saw 8
Skipping line 19789: Expected 5 fields in line 19789, saw 7
Skipping line 19790: Expected 5 fields in line 19790, saw 15
Skipping line 19791: Expected 5 fields in line 19791, saw 9
Skipping line 19792: Expected 5 fields in line 19792, saw 9
Skipping line 19793: Expected 5 fields in line 19793, saw 

In [93]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [94]:
df.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [95]:
df.shape

(20822, 5)

In [96]:
df = df.dropna()

In [97]:
df.tail()

,id,title,author,text,label
20817,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20818,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20819,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20820,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20821,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [98]:
df = df[df['label'] != ' как люди воспринимают своё положение.']

In [99]:
df = df[df['label'] != ' чтобы это была дорога с двусторонним движением.']

In [100]:
df['label'] = df['label'].astype(int)

In [101]:
## Get the independent features

X = df.drop('label', axis=1)

In [102]:
y = df['label']

In [103]:
## Check whether dataset is balanced or not
y.value_counts()

0    10361
1     7922
Name: label, dtype: int64

In [104]:
import tensorflow as tf

In [105]:
tf.__version__

'2.10.0'

In [106]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [107]:
## Vocabulary size
voc_size = 5000

In [108]:
messages = X.copy()

In [109]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [110]:
import nltk
import re 
from nltk.corpus import stopwords

In [111]:
#nltk.download('stopwords')

In [112]:
messages = messages.reset_index()

In [113]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [114]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [115]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]

onehot_repr

[[1808, 3077, 202, 379, 3576, 3949, 1679, 4869, 2580, 2520],
 [2099, 1599, 3645, 2431, 4366, 4988, 4631],
 [276, 4723, 2242, 1022],
 [3475, 2695, 235, 2831, 2287, 1638],
 [3330, 4366, 2925, 2071, 443, 1290, 4366, 4159, 3729, 947],
 [1616,
  1784,
  4821,
  3967,
  1247,
  2160,
  4838,
  1046,
  4225,
  1619,
  1927,
  2786,
  2822,
  289,
  4631],
 [2064, 4115, 3897, 3220, 4124, 2124, 4060, 3456, 4815, 4331, 3619],
 [124, 3075, 686, 514, 3057, 1417, 2160, 2055, 4815, 4331, 3619],
 [3905, 1658, 1459, 1141, 4434, 1082, 3630, 1816, 2160, 1044],
 [4070, 4996, 3863, 335, 3796, 2011, 4739, 1613],
 [1066, 997, 4563, 1783, 2374, 1474, 3804, 192, 356, 1407, 2385],
 [2831, 4314, 3576, 1082, 2160, 3057],
 [3501, 2610, 3401, 3246, 4957, 1719, 3546, 1706, 327],
 [3019, 1880, 1162, 220, 1827, 4822, 3313, 4815, 4331, 3619],
 [4659, 1222, 3259, 1415, 229, 4815, 4331, 3619],
 [3532, 1695, 109, 2809, 20, 725, 2974, 3908, 3404, 277],
 [2845, 2345, 1599],
 [2124, 3829, 1634, 1386, 2160, 2889, 21, 4631],


In [116]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

embedded_docs

array([[   0,    0,    0, ..., 4869, 2580, 2520],
       [   0,    0,    0, ..., 4366, 4988, 4631],
       [   0,    0,    0, ..., 4723, 2242, 1022],
       ...,
       [   0,    0,    0, ..., 4815, 4331, 3619],
       [   0,    0,    0, ..., 4349, 1956,  283],
       [   0,    0,    0, ..., 3685, 3784, 1354]])

In [117]:
# Creating Model

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_4 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [118]:
import numpy as np

X_final = np.array(embedded_docs)
y_final = np.array(y)

In [119]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [120]:
y.value_counts()

0    10361
1     7922
Name: label, dtype: int64

In [121]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 11s 20ms/step - loss: 0.2607 - accuracy: 0.8811 - val_loss: 0.1954 - val_accuracy: 0.9208
Epoch 2/10
383/383 [==============================] - 5s 14ms/step - loss: 0.1276 - accuracy: 0.9502 - val_loss: 0.1958 - val_accuracy: 0.9214
Epoch 3/10
383/383 [==============================] - 5s 14ms/step - loss: 0.0848 - accuracy: 0.9696 - val_loss: 0.2121 - val_accuracy: 0.9200
Epoch 4/10
383/383 [==============================] - 5s 14ms/step - loss: 0.0547 - accuracy: 0.9806 - val_loss: 0.2518 - val_accuracy: 0.9171
Epoch 5/10
383/383 [==============================] - 5s 13ms/step - loss: 0.0363 - accuracy: 0.9869 - val_loss: 0.3327 - val_accuracy: 0.9173
Epoch 6/10
383/383 [==============================] - 5s 14ms/step - loss: 0.0212 - accuracy: 0.9927 - val_loss: 0.3956 - val_accuracy: 0.9100
Epoch 7/10
383/383 [==============================] - 6s 14ms/step - loss: 0.0118 - accuracy: 0.9969 - val_loss: 0.4477 - val_accuracy: 0.916

In [122]:
y_pred = model.predict(X_test)

189/189 [==============================] - 1s 4ms/step


In [123]:
y_pred = np.where(y_pred>=0.5, 1, 0)

In [125]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test, y_pred)

array([[3132,  283],
       [ 245, 2374]], dtype=int64)

In [126]:
print(accuracy_score(y_test,y_pred))

0.9124958568114021
